In [1]:
from pathlib import Path
import pandas as pd
import xlwings as xw
import numpy as np
import matplotlib.pyplot as plt
from rdp import rdp

In [2]:
#Access input folder
input_dir1 = Path ("G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

# Create output directory
output_dir1 = Path("G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_reduced2")
output_dir1.mkdir(exist_ok = True)

# Define current work directory
current_dir = Path.cwd()

# Access the template to be used to generate files
excel_template = current_dir/ "Template.xlsx"

# Mapping dictionary
mapping = {"1": 0, "2": 15, "3": 30, "4": 45, "5": 60}

1 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_section_datasets_issues
2 [WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr15m'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr17m'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr19m'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr21m'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr23m'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr25m'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr27m'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr29m'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_section_datasets_issues/RP_curves_fr30m')]


In [3]:
def reduce_points(time, pressure, file1):
    
    zero_crossing_index = next((i for i, val in enumerate(pressure[:], start=0) if val <= 0), None)
    
    # Combine time and pressure into a single array of 2D points
    points = np.column_stack((time, pressure))
    
    segment1 = points[:zero_crossing_index]  # Example segmentation
    segment2 = points[zero_crossing_index:]
    
    reduced_segment1 = rdp(segment1, epsilon=0.001)
    reduced_segment2 = rdp(segment2, epsilon=0.01)
    
    reduced_points = np.vstack((reduced_segment1, reduced_segment2))
    
    # Separate the reduced points into time and pressure
    reduced_time = reduced_points[:, 0]
    reduced_pressure = reduced_points[:, 1]
    
    # Plot the original and reduced data
    plt.figure(figsize=(4, 4))
    plt.plot(time, pressure, label='Original Data', alpha=0.5)
    plt.plot(reduced_time, reduced_pressure, label='Reduced Data', marker='o')
    plt.xlabel('Time')
    plt.ylabel('Pressure')
    plt.legend()
    
    # Save the plot as an image
    plt.savefig(output_dir3/f'{file1.name}_reduced.png', dpi=150, bbox_inches='tight', facecolor='white')
        
    # Close the plot to free up memory
    plt.close()
    
    return reduced_time, reduced_pressure

In [4]:
def get_phase_array(time, pressure, CT, CP, TT, TP):
    a=0
    b=0
    phase = []
    for t, p in zip(time, pressure):
        if round (t, 10) < round (CT, 10):
            phase.append('positive')#{t} {change_time}')
        elif round (CT, 10) <= round (t, 10) < round(TT, 10):
            a+=1
            if a == 1 and abs(p - CP) <0.000001:
                phase.append('negative_increasing')
            elif a == 1 and abs(p - CP) >0.000001:
                phase.append('error')
                print (f'error- p={p} , change p ={CP}, t ={t}, change t = {CT}')
            else:
                phase.append('negative_increasing')# {t} {trough_time}')
    
        elif round (TT, 10) <= round (t, 10):
            b+=1
            if b == 1 and abs(p - TP) < 0.000001:
                phase.append('negative_decreasing')
            elif b == 1 and abs(p - TP) > 0.000001:
                phase.append('error')
                print (f'error- p={p} , trough p ={TP}, t ={t}, trough t = {TT}')
            else:
                phase.append('negative_decreasing')
        else:
            phase.append('something wrong')
            print ('something wrong')
                
    return phase

In [5]:
def find_CTE(reduced_pressure, reduced_time, file1, EM, SD, A):
    
    lowest_index2 = np.argmin(reduced_pressure[:])
    # TROUGH
    TT = reduced_time[lowest_index2]
    TP = reduced_pressure[lowest_index2]
    
    # Find the first zero crossing between peak and lowest value
    zero_crossing_index_1 = next((i for i, val in enumerate(reduced_pressure[:lowest_index2], start=0) if val <= 0), None)
    
    # CHANGE AND END
    CT = reduced_time[zero_crossing_index_1]
    CP = reduced_pressure[zero_crossing_index_1]
    ET = reduced_time[-1]
    EP = reduced_pressure[-1]
    
    #print ('CT', CT, type(CT), 'CP', CP, type(CP), 'TT', TT, type(TT), 'TP', TP, type(TP))
    print (zero_crossing_index_1, lowest_index2)
    
    if all(isinstance(val, np.floating) for val in [CT, CP, TT, TP]):
    
        phase = get_phase_array(reduced_time, reduced_pressure, CT, CP, TT, TP)
   
        print('phase len', len(phase))
    
        # Create a dictionary with the arrays
        data = {"Mass": [EM] * len(reduced_time),
                 "Standoff distance": [SD] * len(reduced_time),
                 "Angle": [A] * len(reduced_time),
                 'Time': reduced_time,
                 "Phase": phase,
                 'Pressure': reduced_pressure}
        
        # Create DataFrames from dictionaries
        df2 = pd.DataFrame(data)
        
        # Initiate xlwings library
        with xw.App (visible = False) as app:
            
            wb = app.books.open(excel_template)
            # Write dataframe in excel template
            wb.sheets[0].range("A1").options(index=False).value = df2
            
            # Create files in output directory
            wb.save(output_dir3/f"{file1.name}.xlsx")
            wb.close()
        
        print ("file_name",file1.name)
    else:
        CT = CT if isinstance(CT, (np.floating, float)) else None
        CP = CP if isinstance(CP, (np.floating, float)) else None
        TT = TT if isinstance(TT, (np.floating, float)) else None
        TP = TP if isinstance(TP, (np.floating, float)) else None
    
    return TT, TP, CT, CP, ET, EP

In [6]:

df_all = pd.DataFrame(columns=["Mass", "Standoff distance", "Angle", "Arrival time", "AP", "Change time", "CP", "Trough time", "TP", "End time", "EP","Original", "Reduced" ])

for folder1 in input_dir2: # folder1-RP_curves_fr05m

    input_dir3 =  [folder2 for folder2 in folder1.iterdir() if folder2.is_dir()]
    # Create output folders
    output_dir2 = output_dir1/ folder1.name
    output_dir2.mkdir(exist_ok = True)
    print ("OUTPUT 2", output_dir2)
    
    for folder2 in input_dir3: # folder2-05m00.5kg
        # SD AND EM
        SD = int(folder2.name[0:2])
        EM = float(folder2.name[3:-2])
        
        # Create output folders
        output_dir3 = output_dir2/ folder2.name
        output_dir3.mkdir(exist_ok = True)
        print ("OUTPUT 3", output_dir3)
        
        # Make a list of data file names
        excel_files = list(folder2.rglob("*.xlsx"))
        
        # Access each file 
        for file1 in excel_files: # file-1.xlsx
            
            # Assign value to A based on file.stem
            file_name = file1.stem
            A = mapping.get(file_name, None)
            
            # Read data from Excel file
            df1 = pd.read_excel(file1)
            
            # Extract time and pressure columns
            time = df1["Time"].dropna().values
            pressure = df1["Pressure"].dropna().values

            # ARRIVAL
            AT = time[0]
            AP = pressure[0]
                    
            reduced_time, reduced_pressure = reduce_points(time, pressure, file1)
    
            TT, TP, CT, CP, ET, EP = find_CTE(reduced_pressure, reduced_time, file1, EM, SD, A)
                

            # Creating a new DataFrame to append
            row_all = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A],
                                    "Arrival time": [AT],"AP": [AP],
                                    "Change time": [CT], "CP": [CP],
                                    "Trough time": [TT], "TP": [TP],
                                    "End time": [ET], "EP": [EP],
                                    "Original":[len(pressure)], "Reduced": [len(reduced_pressure)]})
            df_all = pd.concat([df_all, row_all], ignore_index=True)

            # Create the plot
            plt.figure(figsize=(4, 4))
            plt.plot(time, pressure, color='b', linestyle=':')
            plt.plot(reduced_time, reduced_pressure, color='r', linestyle='-', marker='o')
            # Save the plot as an image
            plt.savefig(output_dir3/f'{file1.name}.png', dpi=150, bbox_inches='tight', facecolor='white')
                
            # Close the plot to free up memory
            plt.close()                 

OUTPUT 2 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_section_datasets_reduced2\RP_curves_fr15m
OUTPUT 3 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_section_datasets_reduced2\RP_curves_fr15m\15m00.5kg
42 43
phase len 53
file_name 1.xlsx.xlsx
100 101
phase len 110
file_name 2.xlsx.xlsx
60 62
phase len 73
file_name 3.xlsx.xlsx
68 70
phase len 74
file_name 4.xlsx.xlsx
OUTPUT 2 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_section_datasets_reduced2\RP_curves_fr17m
OUTPUT 3 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_section_datasets_reduced2\RP_curves_fr17m\17m00.5kg
43 50
phase len 56
file_name 1.xlsx.xlsx
107 108
phase len 117
file_name 2.xlsx.xlsx
75 78
phase len 86
file_name 3.xlsx.xlsx
69 70
phase len 74
file_name 4.xlsx.xlsx
OUTPUT 2 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_section_datasets_reduced2\RP_curves_fr19m
OUTPUT 3 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_section_datasets_reduced2\RP_curves_fr19m\19m00.5kg
34 40
phase len 42
file_name 1.xlsx.xlsx
76 77
phase len 79
file_name 2.xlsx.xlsx
61 64
phase len 66
file_name 3.x

In [7]:
# Write dataframes in excel template
all_path = 'RP_far_field_all_reduced3.xlsx'

df_all.to_excel(all_path, index=False)